In [ ]:
# IDEA COMPARE VALIDATION METRIC BETWEEN:
# - BASELINE: AUGMENT ALL DATA KEEPING SAME BALANCE BY 10%
# - BETTER: AUGMENT HARDEST 10% BY 100%
# THEN REPEAT SAME STUDY WITH WORST 5%, WORST 15%

In [2]:
import numpy as np
import pandas as pd

import sys, os
sys.path.append(os.path.join(os.getcwd(), '..'))

from src.utils.train_xgb import train_xgb

X_train = np.load('../data/processed/train.npz', allow_pickle=True)['x']
y_train = np.load('../data/processed/train.npz', allow_pickle=True)['y']
X_v = np.load('../data/processed/v.npz', allow_pickle=True)['x']
y_v = np.load('../data/processed/v.npz', allow_pickle=True)['y']

In [4]:
from imblearn.over_sampling import SMOTE

current_class_distribution = pd.Series(y_train).value_counts()
majority_class = current_class_distribution.idxmax()
minority_class = current_class_distribution.idxmin()

sampling_strategy = {
    majority_class: int(current_class_distribution[majority_class] * 1.1),
    minority_class: int(current_class_distribution[minority_class] * 1.1)
}

smote = SMOTE(sampling_strategy=sampling_strategy) 
X_smote, y_smote = smote.fit_resample(X_train, y_train)

In [6]:
mean_base, std_base = train_xgb(X_smote, y_smote, X_v, y_v, n=5)

In [10]:
knn_5 = pd.read_csv('../outputs/results/knn_5.csv')['0']
ind_knn_5 = list(knn_5.sort_values(ascending=True).index)
X_hard = X_train[ind_knn_5[:(len(y_train) // 10)]]
y_hard = y_train[ind_knn_5[:(len(y_train) // 10)]]
X_rest = X_train[ind_knn_5[(len(y_train) // 10):]]
y_rest = y_train[ind_knn_5[(len(y_train) // 10):]]

current_class_distribution = pd.Series(y_hard).value_counts()
majority_class = current_class_distribution.idxmax()
minority_class = current_class_distribution.idxmin()

sampling_strategy = {
    majority_class: int(current_class_distribution[majority_class] * 2),
    minority_class: int(current_class_distribution[minority_class] * 2)
}

smote = SMOTE(sampling_strategy=sampling_strategy) 
X_smote_hard, y_smote_hard = smote.fit_resample(X_hard, y_hard)

X_smote_hard = np.vstack((X_rest, X_smote_hard))
y_smote_hard = np.concatenate((y_rest, y_smote_hard))

In [13]:
mean_hard, std_hard = train_xgb(X_smote_hard, y_smote_hard, X_v, y_v, n=5)

In [14]:
mean_base

0.9198622376502286

In [15]:
mean_hard # MUCH WORSE!

0.9186758648040693

In [16]:
# DID WE SEE ANY IMPROVEMENT?
mean, std = train_xgb(X_train, y_train, X_v, y_v, n=5)
mean

0.9197627231724054